# Desarrollo de las funciones de la API.

In [3]:
import pandas as pd

In [4]:
# @app.get("/")
def peliculas_idioma(idioma:str):
    idioma = idioma.lower()
    df = pd.read_csv('clean_data/movies.csv')
    dfx = pd.read_csv('clean_data/lenguajes.csv')
    dfy = pd.read_csv('clean_data/leng_movies.csv')
    df = df[df.original_language == idioma]
    df = df.merge(dfy,how='left')
    df = df.merge(dfx,how='left')
    abrev = idioma
    idioma = df.lenguaje[0]
    return print(f'La cantidad de películas producidas originalmente en "{idioma} ({abrev})" son: {len(df)}')

# @app.get("/")
def peliculas_duración(pelicula:str):
    pelicula = pelicula.title()
    df = pd.read_csv('clean_data/movies.csv')
    row = df[df.title == pelicula]
    duracion = row.runtime.item()
    anio = row.release_year.item()
    return print(f'La pelicula {pelicula} tiene una duración de {duracion} minutos. Año {anio}')

# @app.get("/")
def franquicia(franquicia:str):
    df = pd.read_csv('clean_data/franquicias.csv')
    dfx = pd.read_csv('clean_data/franq_movies.csv')
    dfy = pd.read_csv('clean_data/movies.csv')
    df = df[df.franquicia == franquicia]
    df = df.merge(dfx)[['id_peli','franquicia']]
    df = df.merge(dfy)[['title','franquicia','revenue','budget']]
    df['ganancia'] = df.revenue - df.budget
    g_total = df.ganancia.sum()
    g_promedio = df.ganancia.mean()
    peliculas = len(df)
    return print(f'La franquicia {franquicia} tiene un total de {peliculas} peliculas, una ganancia total de {g_total} y una ganancia promedio de {g_promedio}') 

# @app.get("/")
def peliculas_pais(pais:str):
    df = pd.read_csv('clean_data/paises.csv')
    dfx = pd.read_csv('clean_data/pais_movies.csv')
    df = df.merge(dfx,how='right')[['id_peli']]
    peliculas = len(df[df.pais == pais])
    return print(f'Se producieron {peliculas} peliculas en el pais {pais}')

# @app.get("/")
def productoras_exitosas(productora:str):
    df = pd.read_csv('clean_data/productoras.csv')
    dfx = pd.read_csv('clean_data/prod_movies.csv')
    dfy = pd.read_csv('clean_data/movies.csv')
    df = df[df.productora == productora]
    df = df.merge(dfx)[['id_peli','productora']]
    df = df.merge(dfy)[['productora','revenue']]
    peliculas = len(df)
    rev_total = df.revenue.sum()
    return print(f'La productora {productora} tiene {peliculas} peliculas con un revenue total de {rev_total} entre todas')

# @app.get("/")
def get_director(director:str):
    df = pd.read_csv('clean_data/directores.csv')
    dfx = pd.read_csv('clean_data/dir_movies.csv')
    dfy = pd.read_csv('clean_data/movies.csv')
    df = df[df.director == director]
    df = df.merge(dfx)[['id_peli','director']]
    df = df.merge(dfy)[['title','release_date','revenue','budget','return']]
    retorno = df.revenue.sum()/df.budget.sum()
    listas = df.to_numpy().tolist()
    return print(f'El director {director} tiene un éxito medido a través del retorno de todas sus películas (esto es: df.revenue.sum()/df.budget.sum()) de {retorno}.\n\nLas películas que dirigió son:\n\n{listas}', "Se utilizó el siguiente calculo para determinar el retorno: df.revenue.sum()/df.budget.sum()")

# @app.get("/")
def recomendacion(pelicula:[str,int]):
    df = pd.read_csv('clean_data/movies_model.csv')
    dfx = pd.read_csv('clean_data/recommended_movies.csv')
    if type(pelicula) == str:
        pelicula = pelicula.title()
        lista = list(dfx[dfx.title == pelicula].idx_recommend)
    elif type(pelicula) == int:
        lista = list(dfx[dfx.id_peli == pelicula].idx_recommend)
        pelicula = dfx.title[dfx.id_peli == pelicula][:1].item()

    recomendacion = df.loc[lista][['id_peli', 'title']]
    recomendacion.columns = ['id_peli','recomendaciones']

    if not recomendacion.empty:
        return {
            'Ingresaste la película: ': f'{pelicula}',
            'Las recomendaciones son: ': f'1.- [{recomendacion.recomendaciones[:1].item()}]; 2.- [{recomendacion.recomendaciones[1:2].item()}]; 3.- [{recomendacion.recomendaciones[2:3].item()}]; 4.- [{recomendacion.recomendaciones[3:4].item()}]; 5.- [{recomendacion.recomendaciones[4:5].item()}]',
        }
    else: 
        return {'Dentro del dataset no se encuentra la película: ' : f'{pelicula}'}

In [17]:
peliculas_duración('The Doom Generation')

La pelicula The Doom Generation tiene una duración de 80.0 minutos. Año 1995


In [5]:
peliculas_pais('United States Of America')

Se producieron 0 peliculas en el pais United States Of America


In [27]:
df = pd.read_csv('clean_data/movies.csv')
df.release_year.max()

2017

In [17]:
df = pd.read_csv('clean_data/movies.csv')
dfx = pd.read_csv('clean_data/lenguajes.csv')
dfy = pd.read_csv('clean_data/leng_movies.csv')
df = df[df.original_language == 'en']
df = df.merge(dfy,how='left')
df = df.merge(dfx,how='left')
abrev = 'en'
idioma = df.lenguaje[0]
print(f'La cantidad de películas producidas originalmente en "{idioma} ({abrev})" son: {len(df)}')

La cantidad de películas producidas originalmente en "English (en)" son: 41090


In [27]:
df = pd.read_csv('clean_data/paises.csv')
dfx = pd.read_csv('clean_data/pais_movies.csv')
df = df[df.pais == 'United States Of America']
df.merge(dfx,how='right')

,id_pais,pais,abrev,id_peli
0,2,NaN,NaN,11860
1,2,NaN,NaN,1408
2,2,NaN,NaN,902
3,2,NaN,NaN,124626
4,2,NaN,NaN,63076
...,...,...,...,...
26130,156,NaN,NaN,132873
26131,157,NaN,NaN,16403
26132,158,NaN,NaN,277690
26133,159,NaN,NaN,321530


In [30]:
df = pd.read_csv('clean_data/paises.csv')
dfx = pd.read_csv('clean_data/pais_movies.csv')
df = df.merge(dfx,how='right')
peliculas = len(df)
{"Se producieron" : f"{peliculas} peliculas", "en el pais" : f"{'United States Of America'}"}

{'Se producieron': '26135 peliculas', 'en el pais': 'United States Of America'}

In [39]:
len(sorted(df.pais.unique()))

154

In [45]:
dfx[dfx.id_pais]

,id_peli,id_pais


In [43]:
pd.read_csv('clean_data/paises.csv')

,id_pais,pais,abrev
0,1,United States Of America,US
1,2,Germany,DE
2,3,United Kingdom,GB
3,4,France,FR
4,5,Italy,IT
...,...,...,...
155,156,Antarctica,AQ
156,157,Gibraltar,GI
157,158,Brunei Darussalam,BN
158,159,Honduras,HN


In [50]:
pd.read_csv('Data/movies_dataset.csv')[['id','production_countries','title']]

C:\Users\tinma\AppData\Local\Temp\ipykernel_14352\3611600278.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('Data/movies_dataset.csv')[['id','production_countries','title']]


,id,production_countries,title
0,862,"[{'iso_3166_1': 'US', 'name': 'United States o...",Toy Story
1,8844,"[{'iso_3166_1': 'US', 'name': 'United States o...",Jumanji
2,15602,"[{'iso_3166_1': 'US', 'name': 'United States o...",Grumpier Old Men
3,31357,"[{'iso_3166_1': 'US', 'name': 'United States o...",Waiting to Exhale
4,11862,"[{'iso_3166_1': 'US', 'name': 'United States o...",Father of the Bride Part II
...,...,...,...
45461,439050,"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",Subdue
45462,111109,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",Century of Birthing
45463,67758,"[{'iso_3166_1': 'US', 'name': 'United States o...",Betrayal
45464,227506,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",Satan Triumphant


In [52]:
dfx[dfx.id_peli == 862]

,id_peli,id_pais
